In [1]:
import pandas as pd
import numpy as np
import sqlite3
import statsmodels.formula.api as smf
import wrds
from regtabletotext import prettify_result
from dotenv import load_dotenv

from plotnine import *
from mizani.formatters import comma_format, percent_format
from datetime import datetime

In [2]:
from sqlalchemy import create_engine
import os

load_dotenv()

connection_string = (
  "postgresql+psycopg2://"
 f"{os.getenv('WRDS_USER')}:{os.getenv('WRDS_PASSWORD')}"
  "@wrds-pgdata.wharton.upenn.edu:9737/wrds"
)

wrds = create_engine(connection_string, pool_pre_ping=True)

In [3]:
start_date = "01/01/1960"
end_date = "12/31/2023"

In [4]:
crsp_monthly_query = (
  "SELECT msf.permno, date_trunc('month', msf.mthcaldt)::date AS date, "
         "msf.mthret AS ret, msf.shrout, msf.mthprc AS altprc, "
         "ssih.primaryexch, ssih.siccd "
    "FROM crsp.msf_v2 AS msf "
    "INNER JOIN crsp.stksecurityinfohist AS ssih "
    "ON msf.permno = ssih.permno AND "
       "ssih.secinfostartdt <= msf.mthcaldt AND "
       "msf.mthcaldt <= ssih.secinfoenddt "
   f"WHERE msf.mthcaldt BETWEEN '{start_date}' AND '{end_date}' "
          "AND ssih.sharetype = 'NS' "
          "AND ssih.securitytype = 'EQTY' "  
          "AND ssih.securitysubtype = 'COM' " 
          "AND ssih.usincflg = 'Y' " 
          "AND ssih.issuertype in ('ACOR', 'CORP') " 
          "AND ssih.primaryexch in ('N', 'A', 'Q') "
          "AND ssih.conditionaltype in ('RW', 'NW') "
          "AND ssih.tradingstatusflg = 'A'"
)

crsp_monthly = (pd.read_sql_query(
    sql=crsp_monthly_query,
    con=wrds,
    dtype={"permno": int, "siccd": int},
    parse_dates={"date"})
  .assign(shrout=lambda x: x["shrout"]*1000)
)

In [5]:
crsp_monthly = (crsp_monthly
  .assign(mktcap=lambda x: x["shrout"]*x["altprc"]/1000000)
  .assign(mktcap=lambda x: x["mktcap"].replace(0, np.nan))
)

In [6]:
mktcap_lag = (crsp_monthly
  .assign(
    date=lambda x: x["date"]+pd.DateOffset(months=1),
    mktcap_lag=lambda x: x["mktcap"]
  )
  .get(["permno", "date", "mktcap_lag"])
)

crsp_monthly = (crsp_monthly
  .merge(mktcap_lag, how="left", on=["permno", "date"])
)

In [7]:
def assign_exchange(primaryexch):
    if primaryexch == "N":
        return "NYSE"
    elif primaryexch == "A":
        return "AMEX"
    elif primaryexch == "Q":
        return "NASDAQ"
    else:
        return "Other"

crsp_monthly["exchange"] = (crsp_monthly["primaryexch"]
  .apply(assign_exchange)
)

In [8]:
def assign_industry(siccd):
    if 1 <= siccd <= 999:
        return "Agriculture"
    elif 1000 <= siccd <= 1499:
        return "Mining"
    elif 1500 <= siccd <= 1799:
        return "Construction"
    elif 2000 <= siccd <= 3999:
        return "Manufacturing"
    elif 4000 <= siccd <= 4899:
        return "Transportation"
    elif 4900 <= siccd <= 4999:
        return "Utilities"
    elif 5000 <= siccd <= 5199:
        return "Wholesale"
    elif 5200 <= siccd <= 5999:
        return "Retail"
    elif 6000 <= siccd <= 6799:
        return "Finance"
    elif 7000 <= siccd <= 8999:
        return "Services"
    elif 9000 <= siccd <= 9999:
        return "Public"
    else:
        return "Missing"

crsp_monthly["industry"] = (crsp_monthly["siccd"]
  .apply(assign_industry)
)

In [9]:
tidy_finance = sqlite3.connect(database="data/crsp.db.sqlite")
general_db = sqlite3.connect(database="data/db.sqlite")
factors_ff3_monthly = pd.read_sql_query(
  sql="SELECT date, rf FROM factors_ff3_monthly",
  con=general_db,
  parse_dates={"date"}
)
  
crsp_monthly = (crsp_monthly
  .merge(factors_ff3_monthly, how="left", on="date")
  .assign(ret_excess=lambda x: x["ret"]-x["rf"])
  .assign(ret_excess=lambda x: x["ret_excess"].clip(lower=-1))
  .drop(columns=["rf"])
)

In [10]:
crsp_monthly = (crsp_monthly
  .dropna(subset=["ret_excess", "mktcap", "mktcap_lag"])
)

(crsp_monthly
  .to_sql(name="crsp_monthly", 
          con=tidy_finance, 
          if_exists="replace",
          index=False)
)

3378303

In [11]:
ccmxpf_linktable_query = (
  "SELECT lpermno AS permno, gvkey, linkdt, "
         "COALESCE(linkenddt, CURRENT_DATE) AS linkenddt "
    "FROM crsp.ccmxpf_linktable "
    "WHERE linktype IN ('LU', 'LC') "
          "AND linkprim IN ('P', 'C') "
          "AND usedflag = 1"
)

ccmxpf_linktable = pd.read_sql_query(
  sql=ccmxpf_linktable_query,
  con=wrds,
  dtype={"permno": int, "gvkey": str},
  parse_dates={"linkdt", "linkenddt"}
)


In [12]:
ccm_links = (crsp_monthly
  .merge(ccmxpf_linktable, how="inner", on="permno")
  .query("~gvkey.isnull() & (date >= linkdt) & (date <= linkenddt)")
  .get(["permno", "gvkey", "date"])
)

crsp_monthly = (crsp_monthly
  .merge(ccm_links, how="left", on=["permno", "date"])
)

In [13]:
(crsp_monthly
  .to_sql(name="crsp_monthly", 
          con=tidy_finance, 
          if_exists="replace",
          index=False)
)

3378303

In [14]:
tidy_finance.execute("VACUUM")
general_db.execute("VACUUM")

In [15]:
general_db.close()
tidy_finance.close()